https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [172]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [86]:
# load data
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [89]:
# parameters
learning_rate = 1e-2

n_input = x.shape[1]
n_hidden = 20
n_output = np.unique(y).shape[0]

In [115]:
# split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)


# define loader
train = TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long())
train_loader = DataLoader(train, batch_size=20, shuffle=True)

test = TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).long())
test_loader = DataLoader(test, shuffle=True)

In [181]:
# model class definition
class NNModel(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(NNModel, self).__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        x = F.relu(self.l1(x))
        y = self.l2(x)
        return y

In [182]:
# model definition
# model = nn.Sequential(
#     nn.Linear(n_input, n_hidden),
#     nn.ReLU(),
#     nn.Linear(n_hidden, n_output)
# )

model = NNModel(n_input, n_hidden, n_output)

In [176]:
# learning functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [177]:
# train model
def train(epoch):
    total_loss = 0
    total_size = 0
    
    # switch train model mode
    model.train()
    for data, target in train_loader:
        # forward
        y_pred = model(data)

        # compute loss
        loss = criterion(y_pred, target)

        # optimizer
        optimizer.zero_grad()

        # learn
        loss.backward()
        
        total_loss += loss.item()
        total_size += data.size(0)

        # step
        optimizer.step()

    if epoch%100 == 0:
        print(f'{epoch}-epoch: average loss: {total_loss/total_size}')

In [178]:
# test function
def test():
    # switch model test mode
    model.eval()
    
    total = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f'Accuracy: {100*correct/total}%[{correct}/{total}]')

In [179]:
for epoch in range(1000):
    train(epoch+1)
test()

100-epoch: average loss: 0.0041438692825890725
200-epoch: average loss: 0.0060522037669129315
300-epoch: average loss: 0.0032799653553714355
400-epoch: average loss: 0.007809372973583993
500-epoch: average loss: 0.006288190336809272
600-epoch: average loss: 0.005244687235071545
700-epoch: average loss: 0.0038622098841837476
800-epoch: average loss: 0.007990228313775289
900-epoch: average loss: 0.007156315960344814
1000-epoch: average loss: 0.0032692196778953075
Accuracy: 37.77777777777778%[17/45]


In [180]:
# save model
torch.save(model.state_dict(), 'model/model.pth')

In [187]:
# load model
model = NNModel(n_input, n_hidden, n_output)
model.load_state_dict(torch.load('model/model.pth'))

In [186]:
test()

Accuracy: 37.77777777777778%[17/45]


In [198]:
# predict
y_pred = model(torch.Tensor([[1, 0, 0, 1]]))

# show predict result
display(y_pred)
result = torch.max(y_pred, 1)

# get max score class
display(result)

# get predict label
pred_label = result[1].item()
display(pred_label)

tensor([[ 5.0545,  4.2406, -8.6444]], grad_fn=<AddmmBackward>)

(tensor([5.0545], grad_fn=<MaxBackward0>), tensor([0]))

0